In [1]:
import os
transcript_dir = r"..\transcripts"

transcripts = {}
for filename in os.listdir(transcript_dir):
    with open(os.path.join(transcript_dir, filename), 'r') as f:
        transcripts[filename.split(".")[0]] = f.read()

transcripts.keys()

dict_keys(['4tools', 'baselifesciences', 'bookstar', 'calmsea', 'deel', 'explorer', 'keeper', 'lasebane', 'mode', 'ss', 'stackcentral', 'xp'])

In [2]:
labels = {
    "4tools": """Priority: “Understand content workflow with multiple contractors so you can increase your engagement size from 10k to 20k" (see 3:07 and 8:41)
Event: “Trial expires in 8 days” (see 3:07 and 8:41)
Date: <2 weeks from now> (see 43:18)
Message: Hi Ryan, last we spoke, it was important to understand your content workflow across clients so you can scale up your engagement size to 20k. Your trial has ended. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.""",
    
    "bookstar": """Priority: “Traditional outreach is failing and needs to invest in social selling but marketing is stretched thin" (see 2:40)
Event: “Kate to talk to her boss” (see 31:24)
Date: <next June> (see 38:26)
Message: Hi Kate, last we spoke, it was important to understand your content workflow across clients so you can scale up your engagement size to 20k. Your trial expires tomorrow. Wanted to reconnect to see if you’re ready to make a decision to proceed or not.""",
    
    "xp": """Priority: “Get rid of Terraform Cloud in Q3 because support is not great" (see 3:42)
Event: “TCF contracts expires” (see 31:24)
Date: <next December> (see 5:03)
Message: Hi Mike, last we spoke, it was important to move off Terraform Cloud in Q3. I know your contract ends in December. Wanted to reconnect to see if we’re on track to replace TFC.""",
    
    "mode": """Priority: “Increase thought leadership as part of ABM execution" (see 22:42)
Event: “ABM research is complete” (see 9:53)
Date: <next December> (see 5:03)
Message: Hi Swati, hope your conference in Copenhagen went well. When we last spoke, you were wrapping up the ABM account research phase. You mentioned you’d be able to enter the execution phase. Wanted to reconnect to see if you’re ready to improve Jan’s LinkedIn presence to reach key decision makers in November.""",

    }

In [3]:
from dotenv import load_dotenv

load_dotenv("../.env")

True

In [5]:
from langchain_openai import AzureChatOpenAI


lm = AzureChatOpenAI(model="gpt-4o")

In [21]:
from langchain.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough


action_item_system_message = """You are a salesperson who has just finished a call with a client. You need to analyze the sales call transcript and provide hidden action item, if any."""

action_item_user_message = """Analyze the following sales conversation transcript and extract key information for future follow-ups.
Find the action item and when to reach out to the client. Only the action item that is more than 2 weeks away are considered.
You have to provide the most important action item. If there are multiple action items, choose the most important one.
If there are no action items, just say "No action items found."


Focus on events such as:
Availability for future discussions
Contract end dates with competitors
Upcoming product launches or rebrands
Planned purchases of complementary products
Any other time-sensitive opportunities mentioned

Here is the response format:
<event>
Priority: Brief description of the main goal or objective (e.g., contract end, product launch) (timestamp in the call, if available)
Event: Specific future event or deadline (timestamp in the call, if available)
Date: Specific date or general timeframe mentioned (timestamp in the call, if available)
Message: Draft a concise, contextual follow-up message addressing the priority and event
</event>

Here is a sample response:
<event>
Priority: "Increase thought leadership as part of ABM execution" (see 22:42)
Event: "ABM research is complete" (see 9:53)
Date: <next December> (see 5:03)
Message: Hi Swati, hope your conference in Copenhagen went well. When we last spoke, you were wrapping up the ABM account research phase. You mentioned you’d be able to enter the execution phase. Wanted to reconnect to see if you’re ready to improve Jan’s LinkedIn presence to reach key decision makers in November.
</event>

Here is the sales call transcript:
<trascript>
{sales_call_transcript}
</transcript>
"""


action_item_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", action_item_system_message),
        ("human", action_item_user_message),
    ]
)


action_item_chain = action_item_prompt | lm | StrOutputParser()

In [22]:

message_system_message = """You are a salesperson who has to write a follow-up message to a client based on the things discussed in the last call."""

message_user_message = """Using the hidden action items, craft a message to remind the client of the action items and provide the date to reach out to the client.
Draft a concise, contextual follow-up message addressing the priority and event.

Here is a sample:
<sample_item>
Priority: "Increase thought leadership as part of ABM execution" (see 22:42)
Event: "ABM research is complete" (see 9:53)
Date: <next December> (see 5:03)
</sample_item>

<sample_response>
Message: Hi Swati, hope your conference in Copenhagen went well. When we last spoke, you were wrapping up the ABM account research phase. You mentioned you’d be able to enter the execution phase. Wanted to reconnect to see if you’re ready to improve Jan’s LinkedIn presence to reach key decision makers in November.
</sample_response>

Here is the highlighted event for which you need to draft a message:
<event>
{event}
</event>
"""


message_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", message_system_message),
        ("human", message_user_message),
    ]
)

message_chain = message_prompt | lm | StrOutputParser()

In [23]:
inputs = {
    "sales_call_transcript": transcripts["4tools"]
}

result = action_item_chain.invoke(inputs)
print(result)

<event>
Priority: "Evaluation and potential adoption of Letterdrop" (see 43:18)
Event: "Reach out if you haven't heard from me" (see 43:18)
Date: "In two weeks" (see 43:18)
Message: Hi Marc, I hope the trial period with Letterdrop is going well. I wanted to follow up on our last conversation and see if you have any feedback or questions after testing the workflow with your team. Looking forward to hearing your thoughts and discussing the next steps.
</event>


In [24]:
for k,v in transcripts.items():
    inputs = {
        "sales_call_transcript": v
    }

    print(f"{k}: ")
    result = action_item_chain.invoke(inputs)
    print(result)
    print("\n-----------------------------------\n")

4tools: 
<event>
Priority: "Evaluate Letterdrop's suitability for agency use" (see 41:53)
Event: "Test Letterdrop's workflow and integration with clients" (see 40:21)
Date: "Two weeks from today" (see 43:18)
Message: Hi Marc, hope you're doing well. When we last spoke, you mentioned needing about two weeks to test Letterdrop’s workflow and integration with your clients. Just wanted to check in and see how the trial has been going and if there are any questions or feedback you have. Looking forward to hearing from you!
</event>

-----------------------------------

baselifesciences: 
<event>
Priority: "Workshop on the policy library for optimal usage" (see 08:39)
Event: "Scheduling a workshop on the policy library" (see 09:26)
Date: September or October (see 09:26)
Message: Hi Jonathan, I hope you're doing well. As we discussed in our last call, we are looking to schedule a workshop on the policy library to help your team make the most of it. Would September or October work better for y